In [2]:
import cv2 as cv
import os
import numpy as np
import pickle
from time import time

DS_DIRNAME = 'ds'
DS_FINAL_DIRNAME = 'ds_final'

TRAIN_DIRNAME = 'train'
VAL_DIRNAME = 'val'

FRAMES_DIRNAME = 'frames'
LABELS_DIRNAME = 'obj_train_data'

Cut frames from vid

In [37]:
for dirname in os.listdir(DS_DIRNAME):
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue
    
    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    os.makedirs(frames_path, exist_ok=True)

    vid = cv.VideoCapture(vid_path)
    ret, frame = vid.read()
    frame_counter = 0
    while vid.isOpened() and ret:
        frame_jpg_name1 = f'{frames_path}/{frame_counter:06}.jpg'
        frame_jpg_name2 = f'{frames_path}/{dirname}_{frame_counter:06}.jpg'
        if not os.path.isfile(frame_jpg_name1) and not os.path.isfile(frame_jpg_name2):
            cv.imwrite(frame_jpg_name1, frame)
        
        ret, frame = vid.read()
        frame_counter += 1
    vid.release()

Check 15002 frames and labels

In [35]:
VID_LEN = 15002
all_len = []
for dirname in os.listdir(DS_DIRNAME):
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue
    
    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    labels_path = f'{DS_DIRNAME}/{dirname}/{LABELS_DIRNAME}'
    all_len.append(len(os.listdir(frames_path)) == VID_LEN)
    all_len.append(len(os.listdir(labels_path)) == VID_LEN)
all(all_len)


False

Del '.*_' from labels and frames

In [28]:
for dirname in os.listdir(DS_DIRNAME):
    counter = 0
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue

    labels_path = f'{DS_DIRNAME}/{dirname}/{LABELS_DIRNAME}'
    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    for files_dir in [labels_path, frames_path]:
        for name in os.listdir(files_dir):
            if '_' in name:
                counter += 1
                new_name = name.split('_')[-1]
                os.rename(f'{files_dir}/{name}', f'{files_dir}/{new_name}')
    print(f'{dirname} - {counter}')

614394-video.h264.mp4 - 15003
614654-video.h264.mp4 - 15002
617343-video.h264.mp4 - 15002
617356-video.h264.mp4 - 15002
617369-video.h264.mp4 - 15002
617382-video.h264.mp4 - 15002
617421-video.h264.mp4 - 15002
617460-video.h264.mp4 - 15002
617473-video.h264.mp4 - 15002
617486-video.h264.mp4 - 15002
617499-video.h264.mp4 - 15002
617512-video.h264.mp4 - 15002


Detect similar images

In [26]:
img1 = cv.imread(f'ds/1.mp4/frames/000000.jpg', cv.IMREAD_GRAYSCALE)
img2 = cv.imread(f'ds/1.mp4/frames/000001.jpg', cv.IMREAD_GRAYSCALE)
np.sum(cv.absdiff(img1, img2))

303817

In [4]:
with open('to_skip.p', 'rb') as f:
    to_skip_p = pickle.load(f)

In [5]:
len(to_skip_p)

102214

In [22]:
DUPLICATES_MAX_DIFF = 1e7
FRAMES_STEP = 250
to_skip = set()
timer = time()
to_skip_prev = len(to_skip)
for dirname in os.listdir(DS_DIRNAME):
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue

    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    frames = sorted(os.listdir(frames_path))
    for i in range(len(frames) // FRAMES_STEP + 1):
        ref_frame = frames[i*FRAMES_STEP]
        similar_to_ref = set()
        left_border = i*FRAMES_STEP + 1
        right_border = min((i+1)*FRAMES_STEP, len(frames))
        img1 = cv.imread(f'{frames_path}/{ref_frame}', cv.IMREAD_GRAYSCALE)
        for j in range(left_border, right_border):
            img2 = cv.imread(f'{frames_path}/{frames[j]}', cv.IMREAD_GRAYSCALE)
            if np.sum(cv.absdiff(img1, img2)) <= DUPLICATES_MAX_DIFF:
                similar_to_ref.add(f'{frames_path}/{frames[j]}')
        to_skip.update(similar_to_ref)

    print(f'{dirname} - {int(time() - timer)}s - {len(to_skip) - to_skip_prev} to skip')
    to_skip_prev = len(to_skip)

614394-video.h264.mp4 - 657s - 13359 to skip
614654-video.h264.mp4 - 1352s - 13192 to skip
617343-video.h264.mp4 - 1997s - 12659 to skip
617356-video.h264.mp4 - 2645s - 13667 to skip
617369-video.h264.mp4 - 3204s - 12448 to skip
617382-video.h264.mp4 - 3780s - 12554 to skip
617421-video.h264.mp4 - 4352s - 11261 to skip
617460-video.h264.mp4 - 4928s - 11753 to skip
617473-video.h264.mp4 - 5567s - 12703 to skip
617486-video.h264.mp4 - 6230s - 13432 to skip
617499-video.h264.mp4 - 6849s - 13392 to skip
617512-video.h264.mp4 - 7365s - 11720 to skip


In [25]:
len(to_skip)

152140

In [26]:
with open('to_skip.p', 'wb') as f:
    pickle.dump(to_skip, f)

Name labels and frames for extraction into final ds

In [29]:
for dirname in os.listdir(DS_DIRNAME):
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue

    labels_path = f'{DS_DIRNAME}/{dirname}/{LABELS_DIRNAME}'
    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    counter = 0
    for frame in os.listdir(frames_path):
        frame_full_name = f'{frames_path}/{frame}'
        if frame_full_name not in to_skip and '_' not in frame:
            frame_new_name = f'{dirname}_{frame}'
            label = frame.split('.')[0] + '.txt'
            label_new_name = f'{dirname}_{label}'
            os.rename(f'{frames_path}/{frame}', f'{frames_path}/{frame_new_name}')
            os.rename(f'{labels_path}/{label}', f'{labels_path}/{label_new_name}')
            counter += 1
            
    print(f'{dirname} - {counter} frames')

614394-video.h264.mp4 - 1643 frames
614654-video.h264.mp4 - 1810 frames
617343-video.h264.mp4 - 2343 frames
617356-video.h264.mp4 - 1335 frames
617369-video.h264.mp4 - 2554 frames
617382-video.h264.mp4 - 2448 frames
617421-video.h264.mp4 - 3741 frames
617460-video.h264.mp4 - 3249 frames
617473-video.h264.mp4 - 2299 frames
617486-video.h264.mp4 - 1570 frames
617499-video.h264.mp4 - 1610 frames
617512-video.h264.mp4 - 3282 frames


Move frames and labels into train and val in a ratio

In [37]:
RATIO = 0.8
VID_LEN = 15002
frames_amount = 12*VID_LEN - len(to_skip)
counter = 0
counter_prev = 0
train_frames_path = f'{DS_FINAL_DIRNAME}/{TRAIN_DIRNAME}/{FRAMES_DIRNAME}'
train_labels_path = f'{DS_FINAL_DIRNAME}/{TRAIN_DIRNAME}/{LABELS_DIRNAME}'
val_frames_path = f'{DS_FINAL_DIRNAME}/{VAL_DIRNAME}/{FRAMES_DIRNAME}'
val_labels_path = f'{DS_FINAL_DIRNAME}/{VAL_DIRNAME}/{LABELS_DIRNAME}'
for dirname in [train_frames_path, train_labels_path, val_frames_path, val_labels_path]:
    os.makedirs(dirname, exist_ok=True)
for dirname in os.listdir(DS_DIRNAME):
    vid_path = f'{DS_DIRNAME}/{dirname}/{dirname}'
    if not os.path.isfile(vid_path):
        continue

    labels_path = f'{DS_DIRNAME}/{dirname}/{LABELS_DIRNAME}'
    frames_path = f'{DS_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    for frame_name in os.listdir(frames_path):
        if frame_name.startswith(dirname):
            label_name = frame_name[:-4] + '.txt'
            if counter <= frames_amount * RATIO:
                os.rename(
                    f'{frames_path}/{frame_name}',
                    f'{train_frames_path}/{frame_name}'
                )
                os.rename(
                    f'{labels_path}/{label_name}',
                    f'{train_labels_path}/{label_name}'
                )
            else:
                os.rename(
                    f'{frames_path}/{frame_name}',
                    f'{val_frames_path}/{frame_name}'
                )
                os.rename(
                    f'{labels_path}/{label_name}',
                    f'{val_labels_path}/{label_name}'
                )
            counter += 1

    train_or_val = 'train' if counter <= frames_amount * RATIO else 'val'
    print(f'{dirname} - {counter - counter_prev} - {train_or_val}')
    counter_prev = counter


614394-video.h264.mp4 - 1643 - train
614654-video.h264.mp4 - 1810 - train
617343-video.h264.mp4 - 2343 - train
617356-video.h264.mp4 - 1335 - train
617369-video.h264.mp4 - 2554 - train
617382-video.h264.mp4 - 2448 - train
617421-video.h264.mp4 - 3741 - train
617460-video.h264.mp4 - 3249 - train
617473-video.h264.mp4 - 2299 - train
617486-video.h264.mp4 - 1570 - val
617499-video.h264.mp4 - 1610 - val
617512-video.h264.mp4 - 3282 - val


Move back from ds_final

In [36]:
for dirname in [TRAIN_DIRNAME, VAL_DIRNAME]:
    print(dirname)
    frames_path = f'{DS_FINAL_DIRNAME}/{dirname}/{FRAMES_DIRNAME}'
    labels_path = f'{DS_FINAL_DIRNAME}/{dirname}/{LABELS_DIRNAME}'
    for frame_name in os.listdir(frames_path):
        vid_name = frame_name.split('_')[0]
        new_frames_path = f'{DS_DIRNAME}/{vid_name}/{FRAMES_DIRNAME}'
        os.rename(f'{frames_path}/{frame_name}', f'{new_frames_path}/{frame_name}')
    for label_name in os.listdir(labels_path):
        vid_name = label_name.split('_')[0]
        new_labels_path = f'{DS_DIRNAME}/{vid_name}/{LABELS_DIRNAME}'
        os.rename(f'{labels_path}/{label_name}', f'{new_labels_path}/{label_name}')

train
val
